In [1]:
import findspark
findspark.init()

In [50]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F
import pandas as pd

In [5]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("test") \
        .getOrCreate()

In [15]:
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv.gz', compression='gzip', nrows=101)

In [16]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID                int64
DOlocationID                int64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [41]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', LongType(), True), StructField('DOlocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [18]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [21]:
df_spark = spark.read \
        .option('header', 'true') \
        .option('compression', 'gzip') \
        .schema(schema) \
        .csv('fhv_tripdata_2019-10.csv.gz')

In [24]:
df_spark.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [25]:
df_spark.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [29]:
df_spark.repartition(6)

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropOff_datetime: timestamp, PUlocationID: int, DOlocationID: int, SR_Flag: string, Affiliated_base_number: string]

In [32]:
df_spark.repartition(6).write.parquet('homework_data', mode='overwrite')

In [33]:
df_spark_partitioned = spark.read \
                        .parquet("homework_data/*")

In [81]:
df_spark_partitioned.createOrReplaceTempView('taxi_data')

In [46]:
# df_result = spark.sql(
#     """ 
#         SELECT COUNT(*)
#         FROM taxi_data
#         WHERE EXTRACT (DAY FROM pickup_datetime) = 15
#                 AND EXTRACT(MONTH FROM pickup_datetime) = 10;

#     """
# )


df_result = spark.sql(
    """
        SELECT date_trunc('DAY', pickup_datetime) as date, COUNT(*) as total_rides
        FROM taxi_data
        group by 1
        order by date;
    """
)

In [47]:
df_result.show()

+-------------------+-----------+
|               date|total_rides|
+-------------------+-----------+
|2019-10-01 00:00:00|      59873|
|2019-10-02 00:00:00|      68746|
|2019-10-03 00:00:00|      71638|
|2019-10-04 00:00:00|      68227|
|2019-10-05 00:00:00|      52398|
|2019-10-06 00:00:00|      45665|
|2019-10-07 00:00:00|      66137|
|2019-10-08 00:00:00|      64049|
|2019-10-09 00:00:00|      60468|
|2019-10-10 00:00:00|      68559|
|2019-10-11 00:00:00|      67715|
|2019-10-12 00:00:00|      51434|
|2019-10-13 00:00:00|      45900|
|2019-10-14 00:00:00|      52665|
|2019-10-15 00:00:00|      62610|
|2019-10-16 00:00:00|      68156|
|2019-10-17 00:00:00|      67656|
|2019-10-18 00:00:00|      68471|
|2019-10-19 00:00:00|      52530|
|2019-10-20 00:00:00|      48304|
+-------------------+-----------+
only showing top 20 rows



In [62]:
from datetime import datetime

def trip_duration(pickup_time, dropoff_time):

    pickup = datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S')
    dropoff = datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S')

    duration = (dropoff - pickup).total_seconds()
    duration_hours = duration / 3600.0
    return duration_hours

In [63]:
trip_duration(df_spark_partitioned.pickup_datetime, df_spark_partitioned.dropOff_datetime)

TypeError: strptime() argument 1 must be str, not Column

In [64]:
trip_duration_udf = F.udf(trip_duration, returnType= types.FloatType())

In [65]:
df_spark_partitioned.withColumn("trip_duration", trip_duration_udf(df_spark_partitioned.pickup_datetime, df_spark_partitioned.dropOff_datetime)).show()

Py4JJavaError: An error occurred while calling o284.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 53) (192.168.0.104 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(In

In [60]:
df_with_duration.show()

Py4JJavaError: An error occurred while calling o239.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 51) (192.168.0.104 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(In

In [68]:
df_result = spark.sql(
    """
        SELECT *, (unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime)) / 3600 AS trip_duration_hours
        FROM taxi_data
        order by trip_duration_hours desc;
    """
)

In [69]:
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|trip_duration_hours|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------------+
|              B02832|2019-10-28 09:00:00|2091-10-28 09:30:00|         264|         264|   NULL|                B02832|           631152.5|
|              B02832|2019-10-11 18:00:00|2091-10-11 18:30:00|         264|         264|   NULL|                B02832|           631152.5|
|              B02416|2019-10-31 23:46:33|2029-11-01 00:13:00|        NULL|        NULL|   NULL|                B02416|  87672.44083333333|
|     B00746         |2019-10-01 21:43:42|2027-10-01 21:45:23|         159|         264|   NULL|       B00746         |  70128.02805555555|
|              B0292

In [70]:
df_zones = spark.read \
        .option("header", "true") \
        .csv("taxi_zone_lookup.csv")

In [73]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [71]:
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [88]:
df_result_1 = spark.sql(
    """
        SELECT PUlocationID, count(*) as  frequency
        FROM taxi_data
        group by PUlocationID
        order by frequency;
    """
)

In [89]:
df_result_1.show()

+------------+---------+
|PUlocationID|frequency|
+------------+---------+
|           2|        1|
|         105|        2|
|         111|        5|
|          30|        8|
|         120|       14|
|          12|       15|
|         207|       23|
|          27|       25|
|         154|       26|
|           8|       29|
|         128|       39|
|         253|       47|
|          96|       53|
|          34|       57|
|          59|       62|
|          58|       77|
|          99|       89|
|         190|       98|
|          54|      105|
|         217|      110|
+------------+---------+
only showing top 20 rows

